In [11]:
from random import random, seed
from math import exp

##### 1. Initialize Network

In [12]:
def initialize_network(n_inputs, n_hidden, n_outputs):
    # Create a list named 'network' in which I contain hidden and output layer
    network = []
    
    # Create a hidden layer
    hidden_layer = [{'weights' : [random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    
    # Create a output layer
    output_layer = [{'weights' : [random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(output_layer)
    
    return network

##### 2. Forward Propagate Inputs

##### 2-1. Neuron Activation

In [13]:
# Calculate neuron activaton for an input
def activate(weights, inputs):
    # Include the bias for calculation
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation    

##### 2-2. Neuron Transfer

In [14]:
# Transfer neuron activation
def transfer(activation):
    # Sigmoid(Logistic) function is applied
    return 1.0 / (1.0 + exp(-activation))

##### 2-3. Forward Propagate input to a network output

In [16]:
def forward_propagate(network, row):
    # Create a variable 'input' to forward propagate the neurons in the input layer to the hidden layer
    # and save the values in row to the variable 'input'.
    input = row
    
    # Loop the network list to access the layers sequentially
    for layer in network:
        # Create a variable 'new_inputs' to contain the values that result from activate and transfer.
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], input)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
            
        # We need this to be used as the input to the next layer
        input = new_inputs
        
    return network

##### 3. Back Propagate Error

##### 3-1. Transfer Derivative

In [17]:
def transfer_derivative(output):
    # Sigmoid(Logistic) funtion is used.
    return output * (1.0 - output)

##### 3-2. Error Backpropagation

In [21]:
def backward_propagate_error(network, expected):
    for i in range(len(network)-1, -1, -1):
        
        # Start with the output layer because outputs flows backwards from output layer to input layer.
        layer = network[i]   # output layer
        # error_signal = error * transfer_derivative(output) for the output layer
        errors, error_signals = [],[]  
        
        for k, neuron in enumerate(layer):
            if i == len(network) - 1:
                error = neuron['output'] - expected[k]
                error_signal = error * transfer_derivative(neuron['output'])
            else:
                error = 0.0
                for j, next_neuron in enumerate(network[i+1]):
                    error += next_neuron['weights'][k] * next_neuron['delta']
                    error_signal = error * transfer_derivative(neuron['output'])
                    
            errors.append(error)
            error_signals.append(error_signal)
            neuron['delta'] = error_signal

In [22]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
 [{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]

backward_propagate_error(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': 0.0005348048046610517}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': 0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': -0.0771723774346327}]


##### 4. Train Network

In [23]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i, layer in enumerate(network):
        if i == 0:
            inputs = row[:-1]     # last element is the target variable
        else:
            inputs = [neuron['output'] for neuron in network[i-1]]
            
        for neuron in layer:
            for k, input in enumerate(inputs):
                neuron['weights'][k] -= l_rate * neuron['delta'] * input
                neuron['weights'][-1] -- l_rate * neuron['delta']